# Likelihood phylogenetics examples

Probabilistic phylogenetics methods rely on Markov models of sequence substitution. These describe the rates at which the different sequence characters exchange with each other, and define the probabilities with which substitutions occur in a given time period.

Here, we will restrict ourselves to one of the simpler DNA models, the **Kimura Two Parameter model** (often abbreviated to **K80**). Other models of substitution $-$ including protein and codon models $-$ work according to the exact same principles.


### Kimura's two parameter model

This model allows for different substitution rates between *transitions* and *transversions*. Recall that transitions occur between DNA bases of the same chemical type, either purines or pyrimidines. Therefore, C -> T and A -> G substitutions are examples of transitions, whereas G -> T and T -> A are examples of transversions.

In practice the transition and transversion parameters are collapsed into a single parameter, their ratio, $\kappa$, hence the name Kimura's Two Parameter model.

___K80 Instantaneous rate matrix ('Q' matrix)___


| |   T   |   C   |   A   |   G   |
|-|-------|-------|-------|-------|
|**T**|-2-$\kappa$|$\kappa$|1 |   1   |
|**C**|$\kappa$|-2-$\kappa$|1 |   1   |
|**A**|   1   |   1   |-2-$\kappa$|$\kappa$|
|**G**|   1   |   1   |$\kappa$|-2-$\kappa$|


The table above shows the general format of the K80 Q matrix. This example has not been scaled yet to make its units expected substitutions per site. For K80 the appropriate scaling factor is 2 + $\kappa$.


In the next cell there is a function to produce a scaled K80 Q matrix for a given value of $\kappa$. This function can be copied and pasted into an R session. Then it can be used to make a Q matrix with $\kappa=2$ as

    q <- make_k80_q_matrix(2)

In [1]:
# Return an instantaneous rate matrix ('Q' matrix)
# according to the Kimura 2-parameter model
# Example usage: 
#    q <- make_k80_q_matrix(2)
make_k80_q_matrix <- function(kappa) {
    values <- rep(1, 16)
    values[c(2, 5, 12, 15)] <- kappa
    values[c(1, 6, 11, 16)] <- (-2 - kappa)
    m <- matrix(values, nrow = 4, dimnames = list(c("T", "C", "A", "G"),
                                                  c("T", "C", "A", "G")))
    
    # Scale the matrix so the average mutation rate
    # is 1 substitution per site
    m / -mean(diag(m))  # always 2 + kappa for K80 model
}

### Substitution probabilities

We use the Q matrix to calculate the probability of observing a change in a character in a given time period. This is given by the formula

$P = e^{Qt}$

where $e$ is the [matrix exponential function](https://en.wikipedia.org/wiki/Matrix_exponential), and $t$ is a length of time. Unlike parsimony, this accounts for all possible additional unobserved substitutions that may occur between the start and end point of the time period.

##### Identifiability
Although we call $t$ a length of time, what we are really considering is the mutation rate $\times$ time. 
We need an external estimate of mutation rate or some fossil dating information to be able to disentangle the two, as otherwise we can't tell the difference between a high mutation rate acting over a short time, and a low mutation rate acting over a long time.

The next cell provides code to calculate a substitution probability matrix ('P' matrix) from a Q and a length of time. For example, to calculate the substitution probabilities after 0.5 time units, for a Q with $\kappa=1.5$:

    q <- make_k80_q_matrix(1.5)
    p <- make_p_matrix(q, 0.5)

In [2]:
# Return the transition probability matrix
# for a given rate matrix (q_matrix) and
# time interval (t)
make_p_matrix <- function(q_matrix, t) {
    require("Matrix")  # for matrix exponential function, 'expm'
    as.matrix(Matrix::expm(q_matrix * t))
}

### Exercise 1

a) Create a Q matrix with $\kappa=2$. Then, calculate P matrices for a variety of time intervals, ranging from time = 0 to some large number, e.g. 100. 

b) What is the trend in P as the length of time increases?

c) Does the value of $\kappa$ affect this trend?

d) What might the probabilities at time = infinity represent?

# Felsenstein's pruning algorithm

Most of the time in phylogenetics we are interested in the trees implied by sequences, and not so much in probability matrices. The probabilities we have been working with so far can be used to compute the likelihood of a tree, which is the probability of seeing the sequences, given the parameters of the substitution model, the branch lengths and the tree topology. 

To do this properly we need to marginalise over (sum up over) all the possible unobserved ancestral sequences. The efficient way to do this is with **Felsenstein's pruning algorithm**. This is a recursive algorithm that works by calculating the **conditional likelihood vector (CLV)** at each ancestral node on the tree, starting with the ones nearest the tips and working towards the root. 


##### Conditional likelihood vectors

A CLV is a vector of conditional probabilities, with one entry for each possible character. So, for DNA data, each CLV has four entries, corresponding to bases T, C, A and G. The first entry, for 'T', gives the conditional probability of observing the sequence data at all tips below this node, given that the nucleotide at this node is a T.


CLVs for tip nodes are set from the sequence data. For example, the first entry, for 'T', is 1 if the sequence data at this tip is a T, otherwise it is 0. The CLV for 'T' would be $[1~0~0~0]$, and for 'A' would be $[0~0~1~0]$.


___conditional likelihood vector calculation___

To compute the CLV at a node all you need to know is the CLVs for the node's immediate descendents, and the 'P' matrices calculated for the descendant branch lengths. 

$L_{i}(x_{i}) = \sum\limits_{x_{j}}p_{x_{i}x_{j}}(t_{j})~L_{j}(x_{j}) \times \sum\limits_{x_{k}}p_{x_{i}x_{k}}(t_{k})~L_{k}(x_{k})$

There's no getting around that this is complicated, but it reads as

"the i-th CLV entry for base $x_i$ equals the product of a big summation term for the left descendant and another for the right descendant. Each big summation term is the sum over all $j$ possible characters in $x$ (i.e. T,C,A,G) of the substitution probability that base $x_i$ becomes base $x_j$ at the end of branch $j$, times the descendant CLV entry at position $j$. In matrix terms, this is the dot product of the 'P' matrix and the descendant CLV"

##### Overall tree likelihood

Once the CLV at the tree's root has been filled in using the pruning algorithm, the overall likelihood is a weighted sum of the its entries, where the weights, $\pi$, are the equilibrium frequencies for each character as defined by the substitution model. For K80 these are all 0.25.

$L = \sum\limits_{i} \pi_i~L_{root}(x_i)$

The next cell provides some code for computing CLVs and root likelihoods


In [3]:
# Return the conditional likelihood vector for the node
# with left and right descendant CLVs 'left_clv', 'right_clv',
# with branch lengths 'left_branch_length' and 'right_branch_length'
# for a model with rate matrix == 'q_matrix'
# Example:
#     clv_7 <- calculate_clv(make_k80_q_matrix(2), 
#                            0.2, c(1,0,0,0),
#                            0.2, c(0,1,0,0))
calculate_clv <- function(q_matrix, left_branch_length, left_clv, right_branch_length, right_clv) {
    p_left <- make_p_matrix(q_matrix, left_branch_length)
    p_right <- make_p_matrix(q_matrix, right_branch_length)
    p_left %*% left_clv * p_right %*% right_clv
}

# Return the root likelihood (not the log-likelihood)
# for a tree with CLV 'root_clv' at the root, and
# equilibrium base frequencies 'pi'
# Example:
#     lnl <- log(root_likelihood(c(0.002532, 0.001838, 0.000165, 0.00801),
#                                pi = c(0.25, 0.25, 0.25, 0.25)))
root_likelihood <- function(root_clv, pi = NULL) {
    if(is.null(pi)) {
        pi <- rep(1, length(root_clv)) / length(root_clv)
    }
    sum(root_clv * pi)
}

### Exercise 2

The following figure is taken from Chapter 4 of [Computational Molecular Evolution](http://abacus.gene.ucl.ac.uk/CME/) by Ziheng Yang. It shows an example application of the pruning algorithm to calculate the conditional likelihood vectors for a single site on a small example tree. Most of the values have been removed.

a) Try to fill in the missing values in the CLVs. The functions `make_k80_q_matrix` and `calculate_clv` should be all you need. Remember that internal branches ($t_6$, $t_7$ and $t_8$) are length 0.1, external branches ($t_1 - t_5$) are length 0.2, and $\kappa = 2$.

b) What is the likelihood of the tree? The log-likelihood?

c) The values in the conditional likelihood vectors become quite small, even in this toy example. Why might this be a problem, and how could it be avoided?

![Felsenstein's pruning algorithm](yang_fig.png "Felsenstein's pruning algorithm")